In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

random_state = 23523
scoring = 'roc_auc_ovo'

In [8]:
# --- load

X_bus = pd.read_pickle('data/business-classification/X.pkl')
Y_bus = pd.read_pickle('data/business-classification/Y.pkl')

X_wine = pd.read_pickle('data/wine/X.pkl')
Y_wine = pd.read_pickle('data/wine/Y.pkl')

# --- split

X_bus_train, X_bus_test, Y_bus_train, Y_bus_test = train_test_split(
    X_bus.values, Y_bus.values,
    test_size=0.3,
    random_state=random_state
)

X_wine_train, X_wine_test, Y_wine_train, Y_wine_test = train_test_split(
    X_wine, Y_wine,
    test_size=0.3,
    random_state=random_state
)

training_datasets = {
    'BusClass': (X_bus_train, Y_bus_train),
    'Wine': (X_wine_train, Y_wine_train)
}

holdout_datasets = {
    'BusClass': (X_bus_test, Y_bus_test),
    'Wine': (X_wine_test, Y_wine_test)
}

# Step 1.

In [ ]:
# create diagnostics for clustering

In [ ]:
# KMeans
for k in [2, 3, 4]:
    for dataset_name in training_datasets:
        X = training_datasets[dataset_name]
        clustering_model = KMeans(
            n_clusters=k,
            random_state=random_state
        )
        clustering_model.fit(X)
        